**Imports**

In [ ]:
import torch
import h5py
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset,DataLoader, Subset
from sklearn.model_selection import train_test_split
from tqdm import tqdm

classe personnalisé pour le dataset

In [ ]:
class DogDataset(Dataset):
    def __init__(self, h5_file_path, transform=None):
        self.h5_file = h5py.File(h5_file_path, 'r')
        self.images = self.h5_file['images']  # Dataset contenant les images
        self.labels = self.h5_file['labels']  # Dataset contenant les labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx]  # Image brute [H, W, C]
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)  # Appliquer les transformations
        else:
            # Conversion manuelle si aucune transformation n'est définie
            image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1)

        label = torch.tensor(label, dtype=torch.long)
        return image, label

# Définir les transformations
transform = transforms.Compose([
    transforms.ToTensor(),  # Convertit les images [H, W, C] en [C, H, W] et normalise entre 0 et 1
    transforms.Resize((224, 224)),  # Redimensionne les images
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalisation pour ResNet
])

# Charger le dataset
h5_file_path = 'D:/Data/data/dog_dataset_no_aug.h5'

# Initialisation des datasets avec transformations
dataset = DogDataset(h5_file_path, transform=transform)

# Division en ensembles
indices = list(range(len(dataset)))
train_indices, temp_indices = train_test_split(indices, test_size=0.3, random_state=42)
val_indices, test_indices = train_test_split(temp_indices, test_size=0.5, random_state=42)

# Création des DataLoaders
train_loader = DataLoader(Subset(dataset, train_indices), batch_size=64, shuffle=True)
val_loader = DataLoader(Subset(dataset, val_indices), batch_size=64, shuffle=False)
test_loader = DataLoader(Subset(dataset, test_indices), batch_size=64, shuffle=False)

# Vérification des dimensions
for images, labels in train_loader:
    print(f"Batch images shape: {images.shape}")  # Devrait être [batch_size, 3, 224, 224]
    print(f"Batch labels shape: {labels.shape}")
    break

# Sélection des données pour 3 classes
class_indices = [i for i, label in enumerate(dataset.labels) if label in [0, 1, 2]]
subset = Subset(dataset, class_indices)

classe de Early stopping

In [ ]:
class EarlyStopping:
    def __init__(self, patience=5, delta=0):
        self.patience = patience
        self.delta = delta
        self.best_loss = float('inf')
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

Charger le modèle ResNet50 pré-entraîné

In [ ]:
model = models.resnet50(pretrained=True)

# Modifier la couche fully connected pour correspondre au nombre de classes
num_classes = 120  # Nombre de classes dans votre dataset
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Déplacer le modèle sur GPU si disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


Fonction de perte et optimiseur, scheduler pour le learning rate et initialisation de l'early stopping

In [ ]:
criterion = nn.CrossEntropyLoss()  # Fonction de perte
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Optimiseur
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
early_stopping = EarlyStopping(patience=5)

Boucle d’entraînement et de validation

In [ ]:
num_epochs = 10

# Boucle d'entraînement
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        #print(f"Initial shape of images: {images.shape}")
        
        # Réorganisez les dimensions si nécessaire
        if images.shape[-1] == 3:  # Format `[batch_size, height, width, channels]`
            images = images.permute(0, 3, 1, 2)  # Réorganise en `[batch_size, channels, height, width]`

        images, labels = images.to(device), labels.to(device)

        # Conversion des types au besoin
        images = images.float()
        labels = labels.long() 
        
        # Passer les images dans le modèle
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

Scheduler et Early Stopping

In [ ]:
scheduler.step(val_loss)
early_stopping(val_loss)
if early_stopping.early_stop:
    print("Early stopping triggered.")
    break

Validation

In [ ]:
# Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.permute(0, 3, 1, 2).to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    val_loss /= len(val_loader)
    val_acc = 100 * correct / total
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.2f}%")

Sauvegarde du modèle entraîné

In [ ]:
torch.save(model.state_dict(), 'aug_resnet50_dog_breeds.pth')